In [3]:
강남구 = [1168010100,1168010300,1168010400,1168010500,1168010600,1168010700,1168010800,1168011000,1168011100,1168011200,1168011300,1168011400,1168011500,1168011800]
gangnam = ['역삼동','개포동','청담동','삼성동','대치동','신사동','논현동','압구정동','세곡동','자곡동','율현동','일원동','수서동','도곡동']

In [4]:
#필요한 모듈들을 import 합니다.
import requests
from bs4 import BeautifulSoup
import pandas as pd

#변수 리스트 설정(이름,평수,가격,평당가격)
names = []
areas = []
prices = []
area_per_price = []

#강남구에 해당하는 데이터 수집
for area_code in 강남구:
    
    url = 'https://hogangnono.com/region/{}'.format(area_code) #url을 area_code를 이용하여 반복문으로 한번에 모두 가져옵니다.
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    name = []
    for pages in soup.findAll('li'):    # 아파트 이름을 가져옵니다
        for link in pages('h5',{'class':'name'}):
            data = link.text        
            name.append(data)
    area = []    
    for pages in soup.findAll('li'):    # 아프트 면적을 가져옵니다
        for link in pages('span',{'class':'area'}):
            data = link.text        
            area.append(data)
    price = []
    for pages in soup.findAll('li'):    #아파트 가겨을 가져옵니다
        for link in pages('span',{'class':'price'}):
            data = link.text        
            price.append(data)
            
    index_dash = [i for i, n in enumerate(price) if n == '-'] # 아파트 가격이 '-' 로 나와있는 아파트들의 index를 찾아냅니다
    for i in index_dash[::-1]: # '-'로 나와있는 아파트를 리스트에서 지웁니다
        del(name[i])
        del(area[i])
        del(price[i])
    
    new_name = [] # 아파트 이름 앞에 동 이름을 제거합니다
    for k in name:
        k_idx = k.find(' ')
        k = k[k_idx+1:]
        new_name.append(k)
        
    
    int_area = []
    for l in area: # 아파트 면적을 integer로 바꿔줍니다
        per = l[:-1]
        int_area.append(int(per))
    
    new_price = []
    for j in price: # 아파트 가격을 integer 로 바꿔줍니다

        if '억'in j:
            if len(j)>4:
                y_idx = j.find('억')
                pri = j[0:y_idx]
                pri1 = j[y_idx+1:]
                pri2 = pri1.replace(',','')
                j = ((int(pri)*10000)+(int(pri2)))*10000
                new_price.append(j)
            else:
                j = j.replace('억','')
                j = int(j)*100000000
                new_price.append(j)

        else:
            j = j.replace(',','')
            j = int(j)*10000
            new_price.append(j)
            
    per_price = [] # 평당 가격을 계산합니다
    for a in range(len(new_price)):
        perp = new_price[a]/int_area[a]
        per_price.append(round(perp))
        
    
# lists in lists 로 저장합니다
    names.append(new_name)
    areas.append(int_area)
    prices.append(new_price)
    area_per_price.append(per_price)
    
amean = []

for i in range(len(gangnam)): # 동별로 평당가격의 mean을 구해줍니다
    if len(area_per_price[i])>0:
        mean = sum(area_per_price[i])/len(area_per_price[i])
        amean.append(round(mean))
        
    else:
        amean.append(0)
        
# pandas DataFrame을 이용한 시각화
kk = {'dong':강남구,'per price':amean}
df = pd.DataFrame(kk,index = gangnam)
df

,dong,per price
역삼동,1168010100,53803304
개포동,1168010300,52430570
청담동,1168010400,41973418
삼성동,1168010500,38476343
대치동,1168010600,39333962
신사동,1168010700,29941603
논현동,1168010800,27085193
압구정동,1168011000,70984225
세곡동,1168011100,22379926
자곡동,1168011200,21942683


In [5]:
df.to_csv('gangnamgu.csv') # csv파일로 저장합니다